In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numba

from molsim import (
    mullerBrownPotential,
    mullerBrownPotentialAndGradient, 
    plot_muller_brown_heatmap
)

In [ ]:
plot_muller_brown_heatmap()

In [ ]:
@numba.njit
def velocityVerlet(numberOfCycles: int, totalEnergy: float, timeStep: float = 5e-4):
    positions = np.zeros((numberOfCycles, 2), dtype=np.float32)
    energies = np.zeros(numberOfCycles, dtype=np.float32)
    
    positions[0] = np.array([-0.557114228, 1.44889779])
    
    potentialEnergy, gradient_dx, gradient_dy = mullerBrownPotentialAndGradient(positions[0])
    force = -np.array([gradient_dx, gradient_dy])
    
    theta = 2 * np.pi * np.random.rand()
    velocity = np.array([np.cos(theta), np.sin(theta)], dtype=np.float32)
    velocity *= np.sqrt(2 * (totalEnergy - potentialEnergy))
    
    computeKineticEnergy = lambda v: 0.5 * np.sum(v**2)
    energies[0] = potentialEnergy + computeKineticEnergy(velocity)
    
    
    for cycle in range(1, numberOfCycles):
        positions[cycle] = positions[cycle - 1] + velocity * timeStep + 0.5 * force * timeStep**2
        velocity += 0.5 * force * timeStep
        
        potentialEnergy, gradient_dx, gradient_dy = mullerBrownPotentialAndGradient(positions[cycle])
        force = -np.array([gradient_dx, gradient_dy])
        
        velocity += 0.5 * force * timeStep
        
        energies[cycle] = potentialEnergy + computeKineticEnergy(velocity)
    return positions, energies

@numba.njit
def langevin(numberOfCycles: int, temperature: float, timeStep: float = 5e-4):
    positions = np.zeros((numberOfCycles, 2), dtype=np.float64)
    energies = np.zeros(numberOfCycles, dtype=np.float64)
    
    positions[0] = np.array([-0.557114228, 1.44889779])
    
    potentialEnergy, gradient_dx, gradient_dy = mullerBrownPotentialAndGradient(positions[0])
    force = -np.array([gradient_dx, gradient_dy])
    
    theta = 2 * np.pi * np.random.rand()
    velocity = np.array([np.cos(theta), np.sin(theta)], dtype=np.float64)
    velocity *= np.sqrt(temperature)
    
    gamma = 1.0
    theta = np.exp(-gamma * timeStep)
    sigma = np.sqrt((1-theta**2) * temperature)
    
    computeKineticEnergy = lambda v: 0.5 * np.sum(v**2)
    energies[0] = potentialEnergy + computeKineticEnergy(velocity) 
    
    for cycle in range(1, numberOfCycles):
        velocity += 0.5 * force * timeStep
        velocity = theta * velocity + sigma * np.random.randn(2)
        
        positions[cycle] = positions[cycle - 1] + velocity * timeStep
        
        potentialEnergy, gradient_dx, gradient_dy = mullerBrownPotentialAndGradient(positions[cycle])
        force = -np.array([gradient_dx, gradient_dy])
        
        velocity += 0.5 * force * timeStep
        energies[cycle] = potentialEnergy + computeKineticEnergy(velocity)
        
    return positions, energies


## Question 1
Implement Velocity Verlet & plot drift

## Question 2
Plot 8 trajectories at equal temperature and describe anomalies and minimum crossing energy.

## Question 3
Plot the temperature and check if it is in the Maxwell-Boltzmann distribution

## Question 4
Implement Langevin integrator & discuss randomness and conservation laws

## Question 5
Compare and contrast integrators

In [ ]:
pos, energies = langevin(int(1e5), 10.0)
print((energies[-1] - energies[0]) / energies[0])
print(np.mean(energies[100:]))

In [ ]:
fig, ax = plt.subplots(2,4, figsize=(24,10))
ax=ax.flatten()
for i in range(ax.shape[0]):
    plot_muller_brown_heatmap(ax[i])
    pos, energies = langevin(int(1e7), 10.0)
    ax[i].plot(*pos[::10].T, lw=0.5, c='red')
fig.tight_layout()